In [ ]:
import pandas as pd
import logging
from datetime import datetime
from conciliador import conciliar_cielo_erp


# Configuração de logging
logging.basicConfig(
    level=logging.DEBUG,  # ou DEBUG para mais detalhes
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("conciliacao.log", encoding="utf-8"),  # grava em arquivo
        logging.StreamHandler()  # mostra no console
    ]
)

In [ ]:
# Caminhos dos arquivos para teste
caminho_erp = r"C:\Users\yan.fernandes\Documents\Conciliador\Análise de Titulos de Cartão de Terceiros - SFR (1).csv"
caminho_cielo = r"C:\Users\yan.fernandes\Documents\Conciliador\Recebiveis_cielo_detalhe-20250422-20250430-1-1-xlsx.xlsx"

# =========================
# Função de limpeza ERP
# =========================
def limpar_erp(df):
    df["Emissão"] = pd.to_datetime(df["Emissão"], dayfirst=True, errors="coerce")
    parcelas = df["Numero"].str.extract(r"-(\d+)/(\d+)")
    df["Numero da Parcela"] = parcelas[0].astype(float).fillna(1).astype(int)
    df["Total Parcelas"] = parcelas[1].astype(float).fillna(1).astype(int)

    df["Valor"] = (
        df["Valor"].astype(str).str.replace(",", ".", regex=False).astype(float)
    )

    return df

# =========================
# Função de limpeza Cielo
# =========================
def limpar_cielo(df):
    df = df.iloc[8:].reset_index(drop=True)
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    df.columns = df.columns.str.strip().str.lower()

    df = df.rename(columns={
        "valor bruto": "VALOR DA PARCELA",
        "valor líquido": "VALOR LÍQUIDO",
        "número da parcela": "PARCELA",
        "quantidade total de parcelas": "TOTAL_PARCELAS",
        "código da autorização": "AUTORIZAÇÃO",
        "nsu/doc": "NSU/DOC",
        "data da venda": "DATA DA VENDA",
        "data prevista de pagamento": "DATA DE VENCIMENTO"
    })

    for col in ["VALOR DA PARCELA", "VALOR LÍQUIDO"]:
        df[col] = (
            df[col].astype(str).str.replace(",", ".", regex=False).astype(float)
        )

    df["PARCELA"] = pd.to_numeric(df["PARCELA"], errors="coerce").fillna(1).astype(int)
    df["TOTAL_PARCELAS"] = pd.to_numeric(df["TOTAL_PARCELAS"], errors="coerce").fillna(1).astype(int)

    for col in ["DATA DA VENDA", "DATA DE VENCIMENTO"]:
        df[col] = pd.to_datetime(df[col], dayfirst=True, errors="coerce")

    return df

# =========================
# Execução do teste
# =========================
def main():
    # Carrega os arquivos
    df_erp = pd.read_csv(caminho_erp, sep=";", encoding="latin1")  # ou encoding="utf-8" dependendo da origem
    df_cielo = pd.read_excel(caminho_cielo, engine="openpyxl")

    print("✅ Arquivos carregados:")
    print("ERP:", df_erp.shape)
    print("CIELO:", df_cielo.shape)

    # Limpeza
    df_erp = limpar_erp(df_erp)
    df_cielo = limpar_cielo(df_cielo)

    # Conciliação
    df_cielo_conc, df_erp_marcado = conciliar_cielo_erp(df_cielo, df_erp)

    # Resultado
    print(df_cielo_conc[["NSU/DOC", "AUTORIZAÇÃO", "Chave ERP", "Valor ERP", "Status"]].head())

    # Exporta resultados
    df_cielo_conc.to_excel("resultado_conciliacao_cielo.xlsx", index=False)
    df_erp_marcado.to_excel("erp_marcado.xlsx", index=False)
    print("✅ Arquivos salvos.")

if __name__ == "__main__":
    main()